In [1]:
using Pkg
Pkg.activate("/Users/bbrener1/haxx/RNAForecasterPaperCode/")

  Activating project at `~/haxx/RNAForecasterPaperCode`


In [2]:
include("../src/trainRNAForecasterV2.jl")

compressingNetwork (generic function with 1 method)

In [3]:
include("../src/recursivePredictionsV2.jl")

mostTimeVariableGenes

In [4]:
using DelimitedFiles
t0 = transpose(readdlm("sc_velo_t0_emb.tsv"))
t1 = transpose(readdlm("sc_velo_t1_pred_emb.tsv"))
t0 = Float32.(t0)
t1 = Float32.(t1)


2×3696 Matrix{Float32}:
 17.1262   4.8592  18.4263   2.37841  …  9.70007   5.36193  20.2021  20.6939
 11.8578  13.1247  12.0906  16.3903      5.93967  12.9324   15.5233  12.4786

In [5]:
trainedModel,losses = trainRNAForecaster(t0, t1, hiddenLayerNodes = 20,
 batchsize = 200, learningRate = 0.0001, checkStability = false, useGPU = false, nEpochs = 200)


Loading Data...
Training model...
Epoch 1
Loss: 0.2775354
Epoch 2
Loss: 0.2505803
Epoch 3
Loss: 0.226528
Epoch 4
Loss: 0.20502345
Epoch 5
Loss: 0.18574814
Epoch 6
Loss: 0.16841327
Epoch 7
Loss: 0.1527944
Epoch 8
Loss: 0.1386844
Epoch 9
Loss: 0.12590876
Epoch 10
Loss: 0.114325225
Epoch 11
Loss: 0.10379849
Epoch 12
Loss: 0.094226256
Epoch 13
Loss: 0.08551226
Epoch 14
Loss: 0.077572465
Epoch 15
Loss: 0.07033653
Epoch 16
Loss: 0.063740544
Epoch 17
Loss: 0.057728518
Epoch 18
Loss: 0.05224984
Epoch 19
Loss: 0.047257714
Epoch 20
Loss: 0.042709377
Epoch 21
Loss: 0.038570557
Epoch 22
Loss: 0.034803882
Epoch 23
Loss: 0.03137951
Epoch 24
Loss: 0.028267544
Epoch 25
Loss: 0.025442837
Epoch 26
Loss: 0.022879494
Epoch 27
Loss: 0.020555988
Epoch 28
Loss: 0.018451381
Epoch 29
Loss: 0.016547339
Epoch 30
Loss: 0.014826312
Epoch 31
Loss: 0.013272445
Epoch 32
Loss: 0.011871214
Epoch 33
Loss: 0.010609283
Epoch 34
Loss: 0.009474198
Epoch 35
Loss: 0.008454633
Epoch 36
Loss: 0.007540401
Epoch 37
Loss: 0.006721

(NeuralODE(), Float32[0.2775354 0.30016667; 0.2505803 0.27114373; … ; 0.00019034151 0.00020425564; 0.0001895644 0.00020342416])

In [6]:
using JLD2
outModel = cpu(trainedModel)
save_object("pancNODEEmbedded.jld2", Flux.state(outModel))


In [ ]:
# # Let's load a trained model
# using Flux: loadmodel!
# using JLD2
# function loadForecaster(fileName::String, inputNodes::Int, hiddenLayerNodes::Int)
#     #recreate neural network structure
#     nn = Chain(Dense(inputNodes, hiddenLayerNodes, relu),
#                Dense(hiddenLayerNodes, inputNodes))
#     model = NeuralODE(nn, (0.0f0, 1.0f0), Tsit5(),
#                        save_everystep = false,
#                        reltol = 1e-3, abstol = 1e-3,
#                        save_start = false)
#     #load parameters into the model
#     model = loadmodel!(model, load_object(fileName))
#     return model
# end



In [ ]:
# trained = loadForecaster("pancNeuralODEResult.jld2", 1352, 6000)

In [ ]:
# target = Float32.(splicedSub)
# size(target)

In [7]:
include("../src/recursivePredictionsV2.jl")

mostTimeVariableGenes

In [8]:
futures = predictCellFutures(trainedModel,t0,6,damping=1.0f0)

Predicting step 1
Predicting step 2
Predicting step 3
Predicting step 4
Predicting step 5
Predicting step 6


2×3696×6 Array{Float32, 3}:
[:, :, 1] =
 17.166    4.79    18.4654   2.47694  …  9.5478   5.31283  20.2352  20.8382
 12.0848  13.0545  12.1725  16.3469      6.0931  12.8292   15.5496  12.5585

[:, :, 2] =
 17.1055   4.77609  18.4185   2.56012  …  9.44457   5.28193  20.235   20.8657
 12.253   13.0368   12.3472  16.3887      6.18274  12.805    15.6918  12.6801

[:, :, 3] =
 17.0525   4.7624  18.3614   2.64134  …  9.33671   5.25168  20.2401  20.8885
 12.4101  13.0189  12.5341  16.4306      6.27635  12.7806   15.8271  12.8082

[:, :, 4] =
 17.0064   4.74893  18.2947   2.72066  …  9.23406   5.22205  20.25    20.9059
 12.5569  13.0008   12.7322  16.4724      6.36101  12.7561   15.9559  12.9435

[:, :, 5] =
 16.9668   4.73567  18.2332   2.79812  …  9.13679   5.19303  20.2644  20.9176
 12.6942  12.9826   12.9219  16.5143      6.43662  12.7315   16.0788  13.0866

[:, :, 6] =
 16.9331   4.72261  18.1801   2.87379  …  9.04445   5.1646  20.2831  20.9228
 12.8229  12.9643   13.0993  16.5563      6.

In [ ]:
# losses

In [9]:
writedlm("./trajectories_t1.tsv",futures[:,:,1])
writedlm("./trajectories_t2.tsv",futures[:,:,2])
writedlm("./trajectories_t3.tsv",futures[:,:,3])
writedlm("./trajectories_t4.tsv",futures[:,:,4])
writedlm("./trajectories_t5.tsv",futures[:,:,5])
writedlm("./trajectories_t6.tsv",futures[:,:,6])

In [10]:
residual = t1.-t0
sq_res = residual.^2
mean(sq_res)

0.012665073f0

In [11]:
residual = t1.-futures[:,:,1]
sq_res = residual.^2
mean(sq_res)

0.018485762f0